# 读取fpt 生成fpt字典 生成all.txt

In [1]:
import struct, os, json
msgDir = '/home/chasedream1129/Project/chasedream1129/dq7crt_CHS_dev/ROM/ACG/0004000000065E00/MESS/'
STD_unkR = b'TEMP/STEP2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa3\x9e\x92\n'
#index区后40字节区域 所有fpt一致

In [5]:
myAll = ''
myFptD = {}
for fptName in sorted(os.listdir(msgDir)):
    if fptName not in myFptD.keys():
        myFptD[fptName]={}
    with open(os.path.join(msgDir, fptName), 'rb')as f:
        fpt = f.read()
        magic, count, version = struct.unpack_from('4s4xII', fpt)
        p = 0x10
        dataR_start = p+0x20*count+0x40
        unkR = fpt[p+0x20*count:dataR_start]
        for i in range(count):
            txtName, hash, ofs, lenth = struct.unpack_from('16s4sII', fpt, p+0x20*i)
            txtName = txtName.split(b'\x00')[0].decode('ascii')
            msg = fpt[dataR_start+ofs:dataR_start+ofs+lenth].decode('utf-8')
            myFptD[fptName][txtName]=msg
            #if msg[-1]!='\n':
            #print(f'======{name}_BEGIN======\n{msg}\n∅============================')
            myAll += f'======{txtName}_BEGIN======\r\n{msg}\r\n∅============================\r\n'
        #print(fptName, count, version)

In [53]:
with open("./myAll.txt", 'w')as f:
    f.write(myAll)

# 从all.txt读取文本 生成fpt字典

In [54]:
with open('./fpt_list.json', 'r')as f:
    fpt_list = json.loads(f.read())
with open("./myAll.txt", 'r', newline='\r\n')as f:
    myAll = f.read()

In [3]:
import re
def txt2dict(text):
    #regex = r'={6}(?P<id>#\d+.txt)_BEGIN={6}\n(?P<translated>[^∅]+)\n∅'
    regex = r'={6}(?P<id>#\d+.txt)_BEGIN={6}\r\n(?P<translated>[^∅]+)\r\n∅'
    d = {}
    for match in re.finditer(regex, text, 0):
        #id = int(match.group('id'))
        #id = f'#{str(id).zfill(6)}.txt'
        id = match.group('id')
        translated = match.group('translated')
        if id in d.keys():
            #print(int(match.group('id'))//10000*10000)
            d['#'+id]=d[id]
            d[id]=translated
        else:
            d[id]=translated
    return d

In [56]:
allD = txt2dict(myAll)
fptD = {}
for fptName, txt_list in fpt_list.items():
    for txtName in txt_list:
        if fptName not in fptD.keys():
            fptD[fptName]={}
        if fptName in ('#100000.fpt', '#140000.fpt', '#190000.fpt', '#210000.fpt', '#420000.fpt', '#510000.fpt', '#520000.fpt', '#540000.fpt', '#550000.fpt', '#560000.fpt', '#610000.fpt', '#800000.fpt'):
            try:
                msg = allD['#'+txtName]
                #print(fptName)
            except KeyError:
                msg = allD[txtName]
        else:
            msg = allD[txtName]
        fptD[fptName][txtName]=msg

In [57]:
myFptD == fptD

True

# 对比一下hash！

In [58]:
acg_msgDir = '/home/chasedream1129/Project/chasedream1129/dq7crt_CHS_dev/ROM/ACG/0004000000065E00/MESS/'
jpn_msgDir = '/home/chasedream1129/Project/chasedream1129/dq7crt_CHS_dev/ROM/JPN/0004000000065E00/MESS/'

In [59]:
acg_FptD = {}
for fptName in sorted(os.listdir(msgDir)):
    if fptName not in acg_FptD.keys():
        acg_FptD[fptName]={}
    with open(os.path.join(acg_msgDir, fptName), 'rb')as f:
        fpt = f.read()
        magic, count, version = struct.unpack_from('4s4xII', fpt)
        p = 0x10
        dataR_start = p+0x20*count+0x40
        unkR = fpt[p+0x20*count:dataR_start]
        for i in range(count):
            txtName, hash, ofs, lenth = struct.unpack_from('16s4sII', fpt, p+0x20*i)
            txtName = txtName.split(b'\x00')[0].decode('ascii')
            #msg = fpt[dataR_start+ofs:dataR_start+ofs+lenth].decode('utf-8')
            acg_FptD[fptName][txtName]=hash.hex()

In [60]:
jpn_FptD = {}
for fptName in sorted(os.listdir(jpn_msgDir)):
    if fptName not in jpn_FptD.keys():
        jpn_FptD[fptName]={}
    with open(os.path.join(jpn_msgDir, fptName), 'rb')as f:
        fpt = f.read()
        magic, count, version = struct.unpack_from('4s4xII', fpt)
        p = 0x10
        dataR_start = p+0x20*count+0x40
        unkR = fpt[p+0x20*count:dataR_start]
        for i in range(count):
            txtName, hash, ofs, lenth = struct.unpack_from('16s4sII', fpt, p+0x20*i)
            txtName = txtName.split(b'\x00')[0].decode('ascii')
            #msg = fpt[dataR_start+ofs:dataR_start+ofs+lenth].decode('utf-8')
            jpn_FptD[fptName][txtName]=hash.hex()

In [61]:
for fptName, fpt_data in acg_FptD.items():
    for txtName, txt_hash in fpt_data.items():
        if txt_hash != jpn_FptD[fptName][txtName]:
            print(fptName, txtName)

In [62]:
with open('./fpt_hash_list.json', 'w')as f:
    f.write(json.dumps(acg_FptD))

# 对比大成功，那么来生成二进制吧

In [4]:
with open('./fpt_hash_list.json', 'r')as f:
    fpt_hash_list = json.loads(f.read())
acgAllD = txt2dict(myAll)
fptD = {}
b_fpt_dict = {}
for fptName, txt_hash_dict in fpt_hash_list.items():
    b_fptHead = struct.pack('4s4xII', b'FPT0', len(txt_hash_dict), 1) #magic, count, version
    b_fpt_index = b''
    b_fpt_data = b''
    for txtName, txt_hash in txt_hash_dict.items():
        b_txt_hash = bytes.fromhex(txt_hash)
        if fptName not in fptD.keys():
            fptD[fptName]={}
        if fptName in ('#100000.fpt', '#140000.fpt', '#190000.fpt', '#210000.fpt', '#420000.fpt', '#510000.fpt', '#520000.fpt', '#540000.fpt', '#550000.fpt', '#560000.fpt', '#610000.fpt', '#800000.fpt'):
            try:
                msg = acgAllD['#'+txtName]
                #print(fptName)
            except KeyError:
                msg = acgAllD[txtName]
        else:
            msg = acgAllD[txtName]
        b_msg = msg.encode('utf-8')
        b_txtName = txtName.encode('ascii').ljust(0x10, b'\x00')
        b_entry = struct.pack('16s4sII4x', b_txtName, b_txt_hash, len(b_fpt_data), len(b_msg))
        b_fpt_data += b_msg
        b_fpt_index += b_entry
        fptD[fptName][txtName]=(msg, b_txt_hash) 
    b_fpt = b_fptHead + b_fpt_index + STD_unkR + b_fpt_data
    b_fpt_dict[fptName] = b_fpt

NameError: name 'myAll' is not defined

# 测试一下二进制和源文件是否一致

In [8]:
fix_msgDir = '/home/chasedream1129/.local/share/citra-emu/load/mods/0004000000065E00/romfs/MESS/'


In [65]:
for fptName, fpt_bytes in b_fpt_dict.items():
    with open(os.path.join(acg_msgDir, fptName), 'rb')as f:
        raw_fpt = f.read()
    if raw_fpt != fpt_bytes:
        print(fptName)
    with open(os.path.join(fix_msgDir, fptName), 'wb')as f:
        f.write(fpt_bytes)

# 用修改后的all.txt生成二进制

In [6]:
def charReplace(text):
    charReplaceDict = {
    '駁':'潇',
    '駅':'阁',
    '駈':'飓',
    '駐':'摊',
    '駑':'咦',
    '駒':'噢',
    '駕':'贱',
    '駘':'锦',
    '駛':'鲤',
    '駝':'蝾',
    '駟':'螈',
    '駢':'饥',
    '駭':'魇'}
    for old, new in charReplaceDict.items():
        text = text.replace(new, old)
    return text

In [7]:
with open('./fpt_hash_list.json', 'r')as f:
    fpt_hash_list = json.loads(f.read())
with open("/home/chasedream1129/Project/chasedream1129/dq7crt_CHS_dev/git/text/CHS/all.txt", 'r', newline='\r\n')as f:
    fixed_myAll = f.read()
allD = txt2dict(fixed_myAll)
fptD = {}
b_fpt_dict = {}
for fptName, txt_hash_dict in fpt_hash_list.items():
    b_fptHead = struct.pack('4s4xII', b'FPT0', len(txt_hash_dict), 1) #magic, count, version
    b_fpt_index = b''
    b_fpt_data = b''
    for txtName, txt_hash in txt_hash_dict.items():
        b_txt_hash = bytes.fromhex(txt_hash)
        if fptName not in fptD.keys():
            fptD[fptName]={}
        if fptName in ('#100000.fpt', '#140000.fpt', '#190000.fpt', '#210000.fpt', '#420000.fpt', '#510000.fpt', '#520000.fpt', '#540000.fpt', '#550000.fpt', '#560000.fpt', '#610000.fpt', '#800000.fpt'):
            try:
                msg = allD['#'+txtName]
                #print(fptName)
            except KeyError:
                msg = allD[txtName]
        else:
            msg = allD[txtName]
        #msg = msg.replace('今天来这里的事', '咦噢摆摊潇洒地贱卖了飓风阁下房间的锦鲤')
        msg = charReplace(msg) # 临时替换字符来粗暴解决
        b_msg = msg.encode('utf-8')
        b_txtName = txtName.encode('ascii').ljust(0x10, b'\x00')
        b_entry = struct.pack('16s4sII4x', b_txtName, b_txt_hash, len(b_fpt_data), len(b_msg))
        b_fpt_data += b_msg
        b_fpt_index += b_entry
        fptD[fptName][txtName]=(msg, b_txt_hash) 
    b_fpt = b_fptHead + b_fpt_index + STD_unkR + b_fpt_data
    b_fpt_dict[fptName] = b_fpt

In [9]:
for fptName, fpt_bytes in b_fpt_dict.items():
    #with open(os.path.join(acg_msgDir, fptName), 'rb')as f:
    #    raw_fpt = f.read()
    #if raw_fpt == fpt_bytes:
    #    print(fptName)
    with open(os.path.join(fix_msgDir, fptName), 'wb')as f:
        f.write(fpt_bytes)

# 差异行数检查相关

In [10]:
chsAllD = allD

In [11]:
import difflib

In [12]:
c = 0
ac = 0
rc = 0
for k, v in chsAllD.items():
    if acgAllD[k] != v:
        c += 1
        differ = difflib.Differ()
        diff = list(differ.compare(acgAllD[k].split('\r\n'), v.split('\r\n')))
        added_lines = [line for line in diff if line.startswith('+ ')]
        ac += len(added_lines)
        removed_lines = [line for line in diff if line.startswith('- ')]
        rc += len(removed_lines)
        if len(added_lines) != len(removed_lines):
            #print(k, '\n'.join(diff))
            print(k)

print(c, '段')
print(ac, rc, '行')

NameError: name 'acgAllD' is not defined

In [1]:
for k, v in chsAllD.items():
    if len(v.split('\r\n')) != 5:
        print(k)

NameError: name 'chsAllD' is not defined

In [103]:
v.split('\r\n')

['#9999', '残念ですが\u3000制限時間となりました。', 'つづきは\u3000ぜひ\u3000発売後に', 'お楽しみください。', '']